<a href="https://colab.research.google.com/github/OmyyFlow/Optimization/blob/main/Leer_instancias_CVRP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = 'http://vrp.atd-lab.inf.puc-rio.br/media/com_vrp/instances/Vrp-Set-A.zip'
nombre_archivo = 'CVRP_instances.zip'

response = requests.get(url)
with open(nombre_archivo, 'wb') as f:
    f.write(response.content)

print(f"Archivo {nombre_archivo} descargado correctamente")

Archivo CVRP_instances.zip descargado correctamente


In [2]:
!unzip CVRP_instances.zip -d CVRP_instances

Archive:  CVRP_instances.zip
   creating: CVRP_instances/A/
  inflating: CVRP_instances/A/A-n80-k10.sol  
  inflating: CVRP_instances/A/A-n80-k10.vrp  
  inflating: CVRP_instances/A/A-n69-k9.vrp  
  inflating: CVRP_instances/A/A-n69-k9.sol  
  inflating: CVRP_instances/A/A-n65-k9.vrp  
  inflating: CVRP_instances/A/A-n65-k9.sol  
  inflating: CVRP_instances/A/A-n64-k9.vrp  
  inflating: CVRP_instances/A/A-n64-k9.sol  
  inflating: CVRP_instances/A/A-n63-k9.vrp  
  inflating: CVRP_instances/A/A-n63-k9.sol  
  inflating: CVRP_instances/A/A-n63-k10.vrp  
  inflating: CVRP_instances/A/A-n63-k10.sol  
  inflating: CVRP_instances/A/A-n62-k8.vrp  
  inflating: CVRP_instances/A/A-n62-k8.sol  
  inflating: CVRP_instances/A/A-n61-k9.vrp  
  inflating: CVRP_instances/A/A-n61-k9.sol  
  inflating: CVRP_instances/A/A-n60-k9.vrp  
  inflating: CVRP_instances/A/A-n60-k9.sol  
  inflating: CVRP_instances/A/A-n55-k9.vrp  
  inflating: CVRP_instances/A/A-n55-k9.sol  
  inflating: CVRP_instances/A/A-n54-

In [7]:
def leer_demandas_cvrp(nombre_archivo):
    demandas = []
    en_seccion_demandas = False

    with open(nombre_archivo, 'r') as archivo:
        for linea in archivo:
            linea = linea.strip()







            #Buscar el inicio de la sección de demandas
            if linea.startswith("DEMAND_SECTION"):
                en_seccion_demandas = True
                continue


            if en_seccion_demandas:
                if not linea or linea.startswith("EOF") or linea.startswith("DEPOT_SECTION"):
                    break

                #Dividir la línea en ID del nodo y demanda (formato común: "id demanda")
                partes = linea.split()
                if len(partes) >= 2:
                    try:
                        demanda = int(partes[1])  #en segunda columna las demandas
                        demandas.append(demanda)
                    except (ValueError, IndexError):
                        continue

    return demandas



In [8]:
def heuristic_lpt(tasks, m):
    """
    Asigna tareas a máquinas usando la heurística LPT.

    :param tasks: Lista de tiempos/demandas de las tareas (ejemplo: [5, 3, 6, 2, 4]).
    :param m: Número de máquinas/vehículos disponibles.
    :return: Tuple (asignaciones, cargas), donde:
             - asignaciones: Lista de listas con tareas asignadas por máquina.
             - cargas: Lista con la suma de cargas por máquina.
    """
    #ordebar
    sorted_tasks = sorted(tasks, reverse=True)

    #Inicializar máquinas
    loads = [0] * m
    assignments = [[] for _ in range(m)]

    # Asignar cada tarea a la máquina menos cargada
    for task in sorted_tasks:
        # Encontrar la máquina con menor carga
        min_load_idx = loads.index(min(loads))
        assignments[min_load_idx].append(task)
        loads[min_load_idx] += task

    return assignments, loads


# Ejemplo de uso
if __name__ == "__main__":
    tasks = demandas
    m = 5

    assignments, loads = heuristic_lpt(tasks, m)

    print("Asignaciones por máquina:", assignments)
    print("Cargas por máquina:", loads)
    makespan = max(loads)
    print("Makespan:", makespan)

Asignaciones por máquina: [[24, 19, 16, 12, 8, 3], [24, 19, 16, 12, 8, 2, 1], [24, 19, 15, 14, 7, 2, 0], [22, 20, 18, 9, 8, 6], [21, 21, 16, 14, 6, 4]]
Cargas por máquina: [82, 82, 81, 83, 82]
Makespan: 83


In [9]:
def get_Capacity(file_path):
  with open (file_path, "r") as CVRP_file:
    for linea in CVRP_file:
      if linea.startswith("CAPACITY"):
        capacity = int(linea.split(":")[1])
        return capacity

def get_k(file_path):
  bandera = False
  with open(file_path, "r") as CVRP_file:
    for linea in CVRP_file:
      if linea.startswith("COMMENT"):
        for word in linea.split():
          if bandera:
            k = int(word.strip(","))
            return k
          if word == "trucks:":
            bandera = True
            pass


In [10]:
filename = "CVRP_instances/A/A-n80-k10.vrp"
instance = leer_demandas_cvrp(filename)
print(instance)
k = get_k(filename)
print(k)
capacity = get_Capacity(filename)
print(capacity)

[0, 24, 22, 23, 5, 11, 23, 26, 9, 23, 9, 14, 16, 12, 2, 2, 6, 20, 26, 12, 15, 13, 26, 17, 7, 12, 4, 4, 20, 10, 9, 2, 9, 1, 2, 2, 12, 14, 23, 21, 13, 13, 23, 3, 6, 23, 11, 2, 7, 13, 10, 3, 6, 13, 2, 14, 7, 21, 7, 22, 13, 22, 18, 22, 6, 2, 11, 5, 9, 9, 5, 12, 2, 12, 19, 6, 14, 2, 2, 24]
10
100


In [11]:
#procesar todos
import os

base_path = "CVRP_instances/A"
for filename in os.listdir(base_path):
    if filename.endswith(".vrp"):
        file_path = os.path.join(base_path, filename)
        demandas = leer_demandas_cvrp(file_path)
        capacity = get_Capacity(file_path)
        k = get_k(file_path)
        assignments, loads = heuristic_lpt(demandas, m=k)
        makespan = max(loads)

        print(f"Archivo: {filename}")
        print("Asignaciones por máquina:", assignments)
        print("Cargas por máquina:", loads)
        print("Makespan:", makespan)
        print("Capacidad:", capacity)
        print("\n")



Archivo: A-n32-k5.vrp
Asignaciones por máquina: [[24, 19, 16, 12, 8, 3], [24, 19, 16, 12, 8, 2, 1], [24, 19, 15, 14, 7, 2, 0], [22, 20, 18, 9, 8, 6], [21, 21, 16, 14, 6, 4]]
Cargas por máquina: [82, 82, 81, 83, 82]
Makespan: 83
Capacidad: 100


Archivo: A-n48-k7.vrp
Asignaciones por máquina: [[26, 16, 16, 14, 9, 8, 2], [25, 18, 16, 12, 11, 5, 2, 0], [25, 18, 16, 12, 11, 4, 3], [24, 18, 16, 14, 9, 8], [23, 20, 15, 14, 9, 8], [23, 20, 15, 13, 10, 7, 2], [22, 22, 14, 13, 10, 5, 3]]
Cargas por máquina: [91, 89, 89, 89, 89, 90, 89]
Makespan: 91
Capacidad: 100


Archivo: A-n80-k10.vrp
Asignaciones por máquina: [[26, 20, 14, 12, 11, 6, 5], [26, 20, 13, 13, 11, 6, 4, 2], [26, 19, 18, 12, 9, 6, 2, 2], [24, 21, 17, 12, 9, 6, 4, 2], [24, 21, 16, 12, 9, 7, 3, 2], [23, 23, 13, 13, 10, 7, 3, 2], [23, 22, 15, 12, 10, 7, 2, 2, 2], [23, 22, 14, 13, 9, 9, 2, 2], [23, 22, 14, 13, 9, 7, 5, 1], [23, 22, 14, 12, 11, 6, 5, 0]]
Cargas por máquina: [94, 95, 94, 95, 94, 94, 95, 94, 94, 93]
Makespan: 95
Capacida